In [1]:
import pygame
from sys import exit
import math
import random
import numpy as np
import gym
from gym import spaces
import matplotlib.pyplot as plt
import scipy.signal as signal
import os
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

pygame 2.1.0 (SDL 2.0.16, Python 3.9.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Gym

In [2]:
class Ball(pygame.sprite.Sprite):
    def __init__(self,width,height) -> None:
        super().__init__()
        self.image = pygame.image.load('images/football.png').convert_alpha()
        self.image = pygame.transform.scale(self.image,(50,50))
        self.rect = self.image.get_rect(center=(width/2 + 25,height/2 + 25))
    
    def move(self,speed,angle):
        self.rect.x += speed*math.cos(angle/360*2*math.pi)
        self.rect.y += speed*math.sin(angle/360*2*math.pi)

    def update(self,speed,angle):
        self.move(speed,angle)

class Player(pygame.sprite.Sprite):
    def __init__(self,x_pos,y_pos,color='red',speed = 5):
        super().__init__()
        self.image = pygame.Surface((50,50))
        self.image.fill(color)
        self.rect=self.image.get_rect(center=(x_pos,y_pos))
        self.speed = speed
    def update(self):
        return


width=1200
height=705
ACTION_COST,IDEL_COST, GET_BALL, GOAL = -0.1, -0.2, 0.5, 1
RIGHT_ACTION = 0.1
RIGHT_KICK = 0.3
screen = pygame.display.set_mode((width,height), pygame.RESIZABLE)
# clock = pygame.time.Clock()

# Initilize the play field
playground_surface=pygame.image.load('images/playground.png').convert_alpha()
playground_surface=pygame.transform.scale(playground_surface,(width,height))

# Initialize the gate        
goal_surface = pygame.Surface((10,200))
goal_surface.fill('red')
goal_rect = goal_surface.get_rect(center=(width-5,height/2))

# Create a ball instance
ball = pygame.sprite.GroupSingle()
ball.add(Ball(width,height))

# Create playground boundaries
up_surface=pygame.Surface((width,100))
up_rect=up_surface.get_rect(midbottom=(width/2,0))
down_surface=pygame.Surface((width,100))
down_rect=down_surface.get_rect(midtop=(width/2,height))
left_surface=pygame.Surface((100,height))
left_surface.fill('blue')
left_rect=left_surface.get_rect(midright=(0,height/2))
right_surface=pygame.Surface((100,height))
right_rect=right_surface.get_rect(midleft=(width,height/2))

collision_rectangles = [down_rect, up_rect, left_rect, right_rect]
collision_rectangles_surfaces = [down_surface, up_surface, left_surface, right_surface]

# algorithm for boundary collisions
def collision_with_boundary(screen,ball,angle,speed):
    # detec collisions 
    for i in range(len(collision_rectangles)):
        screen.blit(collision_rectangles_surfaces[i],collision_rectangles[i])
        if ball.sprite.rect.colliderect(collision_rectangles[i]):
            angle *= -1 
            angle += np.random.randint(3)-1
            if i > 1:
                speed *= -1
    return angle, speed


class Table_Football_Env(gym.Env):
    global episode_invalid_ops
    def __init__(self,ball,screen,playground,goal,SIZE=(1200,705),):
        # Initialize variables
        self.size   = SIZE
        self.screen = screen
        self.ball   = ball
        self.playground_surface = playground
        self.goal   = goal
        self.speed  = 10
        self.angle  = 90*random.random()-45
        self.done = False
        self.inv_a = 0
        self.valid_actions = []
        # Initialliza football players

        self.players_y  = [np.array([200,350,500]),np.array([125,275,425,575]),np.array([200,350,500]),np.array([125,275,425,575])]
        # self.players_x  = [265,745,985,505]
        self.players_x  = [240,720,960,480]
        self.p_speed = 5
        self.players_groups    = []
        count = 0
        for i,j in zip(self.players_x,self.players_y):
            players_group=pygame.sprite.Group()
            for x in j:
                if count < 2:
                    players_group.add(Player(i,x,'red'))
                else: 
                    players_group.add(Player(i,x,'blue'))            
            self.players_groups.append(players_group)
            count += 1

        # Use for i, gourp in enmuerate(players_groups) to gain access to each group
        # The y location of each group could be referenced as self.players_y[i]
        self._set_world()
        

    def _set_world(self):
        # Genereate player and player moving
        self.players_y  = [np.array([100,350,600]),np.array([125,275,425,575]),np.array([100,350,600]),np.array([125,275,425,575])]
        # initilize ball
        self.angle=90*random.random()-45
        self.speed=10
        self.ball.empty()
        self.ball.add(Ball(width,height))        
        self.done = False      
        self.valid_actions = []
        return self._observe(0)
    
    def reset(self):       
        episode_invalid_ops.append(self.inv_a)
        self.inv_a = 0
        return self._set_world()
    
    def _get_distance_ball_goal(self,obs,x,y):
        # Calculate the distance between the ball and the goal
        # Get indices of cells with value 1
        indices = np.argwhere(obs == 1)
        # Calculate squared distances to (i, j)
        distances_squared = np.sum((indices - np.array([y, x]))**2, axis=1)        
        # Get index of nearest cell
        nearest_index = np.argmin(distances_squared)
        # Calculate distance
        return np.sqrt(distances_squared[nearest_index])

    def _get_distance_ball_player(self,ball,x,y):
        distance = np.sqrt(np.sum((ball[0]-x)**2 + (ball[1]-y)**2))
        return distance
    
    def _observe(self,get):
        field = np.zeros([width,height]).T
        obs = field.copy()
        #  Generate the goal in the observation grid
        obs[int(height/2-100):int(height/2+100),-10:] = 1
        #  Generate the ball in the observation grid
        i ,j = self.ball.sprite.rect.x + 25, self.ball.sprite.rect.y
        ball_location = [i,j]
        distance = self._get_distance_ball_goal(obs,i,j)        
        # Ensure i and j are within valid range when ploting the ball with size
        i = max(24, i)
        j = max(24, j)
        obs[j-24:j+24,i-24:i+24] = 3
        #  Generate the players in the observation grid
        for i,group in enumerate(self.players_groups):            
            x = group.sprites()[0].rect.x + 25            
            if i == 0:
                dis_bp_1=np.array([])
                for j in self.players_y[i]:
                    dis_bp_1 = np.append(dis_bp_1,self._get_distance_ball_player(ball_location,x,j))                    
                    obs[j-24:j+24,x-24:x+24] = 2
            elif i == 1:
                dis_bp_2=np.array([])
                for j in self.players_y[i]:                                        
                    dis_bp_2 = np.append(dis_bp_2,self._get_distance_ball_player(ball_location,x,j))                                        
                    obs[j-24:j+24,x-24:x+24] = 2
            else:
                for j in self.players_y[i]:
                    obs[j-24:j+24,x-24:x+24] = -1           
        return [obs,[distance,dis_bp_1.min(),dis_bp_2.min()],get]

    # Executes an action and returns new state, reward, done, and next valid actions
    def step(self, action,render=False):   
        self.valid_actions = []         
        # exit is user click X
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                exit()

        # define collisions with play yard boundaries
        self.angle, self.speed = collision_with_boundary(self.screen,self.ball,self.angle,self.speed)        
        reward = 0
        get = 0
        # Genereate player and player moving
        for i,group in enumerate(self.players_groups):
        # Collisions detection between ball and players
            if pygame.sprite.spritecollide(self.ball.sprite,group,False):
                if i < 2:
                    reward += GET_BALL
                    get = 1
                    # self._action_kick(i,action)
                    # Adding a new reward for kicking the ball
                    distance = self._observe(get)[1]
                    self._action_kick(i,action)
                    if self._observe(get)[1] < distance:
                      reward += RIGHT_KICK
                else:
                    reward += -GET_BALL
                    self.angle = 90 * random.random() - 45 + 180
        # Execute action
            if i < 2:
                min_dis = self._observe(get)[1][1:]
                self._action_moving(self.players_y[i],group,i,action)
                min_dis_2 = self._observe(get)[1][1:]

            
                if min_dis[0] > min_dis_2[0] or min_dis[1] > min_dis_2[1]:
                    reward += RIGHT_ACTION
            else:
                self.player_automove(self.players_y[i],group)      

        # determine reward
        reward += ACTION_COST
        if self.ball.sprite.rect.colliderect(goal_rect):
            reward += GOAL
        # Reset the game when goal            
            print('goal!') 
            self.reset()
            self.done = True     
        elif action[0] == 0 or action[1] == 0:
            reward += IDEL_COST  


        # render the game if the option is True
        if render:
            self.render()
            
        return self._observe(get),reward,self.done,self.valid_actions,{self.speed,self.angle}
    
    # This function get the position of players and the ball, and update the player position based on the ball location in the y direction
    def player_automove(self,player_y,player_group):
        y = self.ball.sprite.rect.y
        speed_reduction = 4
        y_diff=(player_y-y)[np.argmin(np.abs(player_y-y))]  
        if y_diff > 0 and player_y.min() > 40:
            player_y -= self.p_speed-speed_reduction
        elif y_diff < 0 and player_y.max() < height-65:
            player_y += self.p_speed-speed_reduction
        for sprite, new_y in zip(player_group, player_y):
            sprite.rect.y = new_y

    # This is the action function for ball and blue team player case
    def action_ball(self):
        self.speed *= -1
        self.angle = 90*random.random()-45
        return
        
    def _action_moving(self,player_y,player_group,i,action):        
        if i == 0:
            self.valid_actions.append([0,1,2])
            if action[0] == 1 and player_y.min()> 40:
                player_y -= self.p_speed
            elif action[0] == 2 and player_y.max() < height -65:
                player_y += self.p_speed                
            else:
                action[0] = 0
                if player_y.min()<= 40:
                    self.valid_actions[0]=[0,2]                    
                elif player_y.max() >= height -65:
                    self.valid_actions[0]=[0,1]
                self.inv_a += 1
        if i == 1:
            self.valid_actions.append([0,1,2])
            if action[1] == 1 and player_y.min()> 40:
                player_y -= self.p_speed
            elif action[1] == 2 and player_y.max() < height -65:
                player_y += self.p_speed
            else:
                action[1] = 0
                if player_y.min()<= 40:                    
                    self.valid_actions[1]=[0,2]
                elif player_y.max() >= height -65:                    
                    self.valid_actions[1]=[0,1]                    
                self.inv_a += 1
            
        for sprite, new_y in zip(player_group, player_y):
            sprite.rect.y = new_y

        return

    def _action_kick(self, i, action, action_speed = np.random.randint(5,15), action_angle = 90 * random.random() - 45):        
        if i == 0:
            self.speed = action[2]
            self.angle = action[4]        
        elif i == 1:
            self.speed = action[3]
            self.angle = action[5]        
        else:
            self.speed = action_speed
            self.angle = action_angle        
        return  

    def render(self):
        # Update each element in the play yard
        self.screen.blit(self.playground_surface,(0,0))
        self.screen.blit(self.goal,(width-5,height/2-100))
        
        self.ball.draw(self.screen)
        self.ball.update(self.speed,self.angle)  

        for group in self.players_groups:
            group.draw(self.screen)
            group.update()

        pygame.display.update()
        # clock.tick(60)
        return

# Network

## Network for moving

In [3]:
class NNet():

    def __init__(self, height, width) -> None:
        self.height = height
        self.width = width
        return None
    
    
    def _build_net(self, TRAINING=True):

        action_dim = 2
        # Input for the 2D array
        input_2d = tf.keras.Input(shape=(self.height, self.width, 1), name='input_2d')
        # Input for the single value
        input_single = tf.keras.Input(shape=(3,), name='input_single')
        # Convolutional Neural Network for the 2D array
        conv_layer = layers.Conv2D(32, (5, 5), activation='relu')(input_2d)
        conv_layer = layers.Conv2D(32, (3, 3), activation='relu')(conv_layer)
        maxpool_layer = layers.MaxPooling2D((2, 2))(conv_layer)

        # conv_layer = layers.Conv2D(32, (3, 3), activation='relu')(maxpool_layer)
        conv_layer = layers.Conv2D(16, (3, 3), activation='relu')(conv_layer)
        maxpool_layer = layers.MaxPooling2D((2, 2))(conv_layer)

        # conv_layer = layers.Conv2D(16, (3, 3), activation='relu')(maxpool_layer)
        conv_layer = layers.Conv2D(8, (3, 3), activation='relu')(conv_layer)
        maxpool_layer = layers.MaxPooling2D((2, 2))(conv_layer)


        conv_layer = layers.Conv2D(8, (3, 3), activation='relu')(maxpool_layer)
        maxpool_layer = layers.MaxPooling2D((2, 2))(conv_layer)

        flatten_layer = layers.Flatten()(maxpool_layer)
        # Concatenate the flattened 2D array with the single value input
        concatenated = layers.concatenate([flatten_layer, input_single])
        # Shared Dense layers
        dense1 = layers.Dense(16, activation='relu')(concatenated)
        dense2 = layers.Dense(16, activation='relu', name='dense_layer_2')(dense1)
        # value layer
        value = layers.Dense(1, kernel_initializer='glorot_uniform', name='value_layer')(dense2)
        # The last layer has action_dim * 3 units
        output = layers.Dense(action_dim * 3, name = 'output')(dense2)


        # Reshape to (action_dim, 3) and apply softmax
        output_reshaped = layers.Reshape((action_dim, 3))(output)
        output_softmax = layers.Softmax(axis=-1)(output_reshaped)

        # Create the model
        model = models.Model(inputs=[input_2d, input_single], outputs=[output_softmax,value])

        if TRAINING:
            # Value, Policy, and Validity predictions
            value = layers.Dense(1, name='value')(concatenated)
            policy = layers.Dense(action_dim, activation='softmax', name='policy')(concatenated)
            valids = layers.Dense(action_dim, activation='sigmoid', name='valids')(concatenated)
            
            # Calculate Losses
            value_loss = tf.keras.losses.MeanSquaredError()(self.target_v, value)
            policy_loss = -tf.reduce_sum(tf.log(tf.clip_by_value(policy, 1e-15, 1.0)) * self.advantages)
            entropy_loss = -0.01 * tf.reduce_sum(policy * tf.log(tf.clip_by_value(policy, 1e-10, 1.0)))
            validity_loss = -0.5 * tf.reduce_sum(tf.log(tf.clip_by_value(valids, 1e-10, 1.0)) *
                                                    self.train_valid + tf.log(tf.clip_by_value(1 - valids, 1e-10, 1.0)) * (1 - self.train_valid))

            # Total loss
            loss = value_loss + policy_loss - entropy_loss + validity_loss

            # Optimizer
            optimizer = optimizers.Adam(learning_rate=0.001, clipnorm=1.0)
            model_move.compile(optimizer=optimizer, loss=loss)

        return model
    
    def _build_net_kick(self, TRAINING=True):

        action_dim = 2
        # Input for the 2D array
        input_2d = tf.keras.Input(shape=(self.height, self.width, 1), name='input_2d')
        # Input for the single value
        input_single = tf.keras.Input(shape=(3,), name='input_single')
        # Convolutional Neural Network for the 2D array
        conv_layer = layers.Conv2D(32, (5, 5), activation='relu')(input_2d)
        conv_layer = layers.Conv2D(32, (3, 3), activation='relu')(conv_layer)
        maxpool_layer = layers.MaxPooling2D((2, 2))(conv_layer)

        # conv_layer = layers.Conv2D(32, (3, 3), activation='relu')(maxpool_layer)
        conv_layer = layers.Conv2D(16, (3, 3), activation='relu')(conv_layer)
        maxpool_layer = layers.MaxPooling2D((2, 2))(conv_layer)

        # conv_layer = layers.Conv2D(16, (3, 3), activation='relu')(maxpool_layer)
        conv_layer = layers.Conv2D(8, (3, 3), activation='relu')(conv_layer)
        maxpool_layer = layers.MaxPooling2D((2, 2))(conv_layer)


        conv_layer = layers.Conv2D(8, (3, 3), activation='relu')(maxpool_layer)
        maxpool_layer = layers.MaxPooling2D((2, 2))(conv_layer)

        flatten_layer = layers.Flatten()(maxpool_layer)
        # Concatenate the flattened 2D array with the single value input
        concatenated = layers.concatenate([flatten_layer, input_single])
        # Shared Dense layers
        dense1 = layers.Dense(16, activation='relu')(concatenated)
        dense2 = layers.Dense(16, activation='relu', name='dense_layer_2')(dense1)
        # value layer
        value = layers.Dense(1, kernel_initializer='glorot_uniform', name='value_layer')(dense2)
        # The last layer has action_dim * 3 units
        output_layer = layers.Dense(4, activation='tanh')(dense2)

        # Create the model
        model = models.Model(inputs=[input_2d, input_single], outputs=[output_layer,value])

        if TRAINING:
            # Value, Policy, and Validity predictions
            value = layers.Dense(1, name='value')(concatenated)
            policy = layers.Dense(action_dim, activation='softmax', name='policy')(concatenated)
            valids = layers.Dense(action_dim, activation='sigmoid', name='valids')(concatenated)
            
            # Calculate Losses
            value_loss = tf.keras.losses.MeanSquaredError()(self.target_v, value)
            policy_loss = -tf.reduce_sum(tf.log(tf.clip_by_value(policy, 1e-15, 1.0)) * self.advantages)
            entropy_loss = -0.01 * tf.reduce_sum(policy * tf.log(tf.clip_by_value(policy, 1e-10, 1.0)))
            validity_loss = -0.5 * tf.reduce_sum(tf.log(tf.clip_by_value(valids, 1e-10, 1.0)) *
                                                    self.train_valid + tf.log(tf.clip_by_value(1 - valids, 1e-10, 1.0)) * (1 - self.train_valid))

            # Total loss
            loss = value_loss + policy_loss - entropy_loss + validity_loss

            # Optimizer
            optimizer = optimizers.Adam(learning_rate=0.001, clipnorm=1.0)
            model_move.compile(optimizer=optimizer, loss=loss)

        return model

# Compile the model
master_net = NNet(height,width)
model_move = master_net._build_net(0)
model_move.compile(optimizer='adam', loss='mse')
model_kick = master_net._build_net_kick(0)
model_kick.compile(optimizer='adam', loss='mse')

## Plot the network

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model_kick, to_file='table_football_neural_network_architecture.png', show_shapes=True, show_layer_names=True)

In [ ]:
model_move.summary()

In [4]:
def get_actions(state):
    state[0] = np.expand_dims(state[0], axis=-1)
    state[0] = np.expand_dims(state[0], axis=0)
    state[1] = np.expand_dims(state[1], axis=-1)
    state[1] = np.expand_dims(state[1], axis=0)
    # Use the model to predict the action for this observation
    # print(state[0].shape,state[1],state[1].shape)
    move, value = model_move.predict([state[0],state[1]])
    move = move.reshape((2,3))

    # exploration
    # move = [np.random.choice(3, p=move[i]) for i in range(2)]
    
    # greedy policy
    move = np.argmax(move, axis = 1)    

    # Option one, train the kick model at every state
    # kick = model_kick.predict([state[0],state[1]])    
    # kick[:,0:2] = 3 + ((kick[:,0:2]+1)*(10 - 3)/2)
    # kick[:,2:4] = 0 + ((kick[:,2:4]+1)*(360 - 0)/2)

    # Option two, train the model only when the kick is possible
    if state[2] == 10:
        kick = model_kick.predict([state[0],state[1]])             
        # convert the kick action to kick angle and speed
        kick[:,0:2] = 3 + ((kick[:,0:2]+1)*(10 - 3)/2)
        kick[:,2:4] = 0 + ((kick[:,2:4]+1)*(360 - 0)/2)
    else:
        kick = np.random.rand(4)
        kick[0:2] = 3 + ((kick[0:2])*(10 - 3))
        kick[2:4] = -45 + ((kick[2:4])*(90 - 0))

    return np.append(move,kick),value

# Training loop

In [5]:
def discount(x, gamma):
    return signal.lfilter([1], [1, -gamma], x[::-1], axis=0)[::-1]

def expand_dim(state):
    state = np.expand_dims(state, axis=-1)
    state = np.expand_dims(state, axis=0)    
    return state

In [10]:
import os
# import h5py

# Define a directory to save models
save_dir = 'saved_models'
save_dir_2 = 'saved_images'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
if not os.path.exists(save_dir_2):
    os.makedirs(save_dir_2)

save_interval = 50

# Create the play yard
screen = pygame.display.set_mode((width,height))
# pygame.init()
env = Table_Football_Env(ball=ball,screen=screen,playground=playground_surface,goal=goal_surface)

bootstrap_value = 0
gamma   = .95
rewards = [0]
values = [0]
episodes = 20
batch_size = 32
epochs = 5 

# Shared arrays for tensorboard
episode_rewards        = []
episode_lengths        = []
episode_mean_values    = []
episode_invalid_ops    = []
frame_count = 0

for i_episode in range(episodes):
    valid_actions          = [[0,1,2],[0,1,2]]
    episode_reward = episode_step_count = episode_inv_count = 0
    # Reset the state
    state = env._set_world()
    
    actions = []

    # Lists to store batch data
    batch_states_0 = []
    batch_states_1 = []
    batch_moves = []
    batch_advantages = []

    for i in range(500):
        actions = []
        state[0] = expand_dim(state[0])
        state[1] = expand_dim(state[1])        
        # Get actions and predicted value                
        move, value = model_move.predict([state[0],state[1]])
        move = move.reshape((2,3))
        p = np.random.rand(1)
        if p <= 0.3:
            # exploration
            move = np.random.choice(3, 2)            
        else:
            # greedy policy
            move = np.argmax(move, axis = 1)    

        # Option one, train the kick model at every state
        # kick = model_kick.predict([state[0],state[1]])    
        # kick[:,0:2] = 3 + ((kick[:,0:2]+1)*(10 - 3)/2)
        # kick[:,2:4] = 0 + ((kick[:,2:4]+1)*(360 - 0)/2)

        # Option two, train the model only when the kick is possible
        value_k = np.array([[0]])        
        
        if state[2] == 1:
            kick_k, value_k = model_kick.predict([state[0],state[1]])             
            # convert the kick action to kick angle and speed
            kick = kick_k.copy()
            kick[:,0:2] = 10 + ((kick[:,0:2]+1)*(20 - 10)/2)
            kick[:,2:4] = 0 + ((kick[:,2:4]+1)*(360 - 0)/2)
            print(kick)
        else:
            kick = np.random.rand(4)
            kick[0:2] = 15 + ((kick[0:2])*(25 - 15))
            kick[2:4] = -45 + ((kick[2:4])*(90 - 0))
        
        actions = np.append(move,kick)        


        for i in [0,1]:
            if actions[i] not in valid_actions[i]:
                actions[i] = random.choice(valid_actions[i])

        # Execute action and observe
        next_state, reward, done, valid_actions, info = env.step(actions,render=True)
        # next_state, reward, done, valid_actions, info = env.step(actions,render=False)
        
        # Save data for the batch
        batch_states_0.append(state[0])
        batch_states_1.append(state[1])
        batch_moves.append(tf.one_hot(move[:2], 3))
        rewards.append(rewards[-1] + reward)
        values.append(values[-1] + value[0,0] + value_k[0,0])          

        # Here we take the rewards and values from the rollout, and use them to 
        # generate the advantage and discounted returns. (With bootstrapping)
        # The advantage function uses "Generalized Advantage Estimation"
        rewards_plus = np.asarray(rewards + [bootstrap_value])
        discounted_rewards = discount(rewards_plus,gamma)[:-1]
        
        value_plus = np.asarray(values + [bootstrap_value])
        advantages = rewards + gamma * value_plus[1:] - value_plus[:-1]
        advantages = discount(advantages,gamma)

        # value_plus = np.expand_dims(value_plus, axis=-1)
        # value_plus = np.expand_dims(value_plus, axis=0)
        # Debugging information to check shapes

        advantages = np.expand_dims(advantages, axis = 0)

        
        move = tf.one_hot(actions[:2],3)
        move = np.expand_dims(move, axis=0)


        # print(state[0].shape)

        # Train the model with the updated Q-value
        model_move.fit([state[0],state[1]], [move,advantages])     

        if state[2] == 1:
            model_kick.fit([state[0],state[1]], [kick_k,advantages])
        
        # Update state
        state = next_state

        # Save the current frame as an image
        if i_episode <= 1:
            pygame.image.save(screen, os.path.join(save_dir_2, f'frame_{frame_count}.png'))
            frame_count += 1

        # Save the model
        # if i % save_interval == 0 and i > 0:
            # model_move.save(os.path.join(save_dir, f'model_move_episode_{i_episode}.h5')) 
            # model_kick.save(os.path.join(save_dir, f'model_kick_episode_{i_episode}.h5')) 
        if done:           
            # model_move.save(os.path.join(save_dir, f'model_move_episode_{i_episode}.h5')) 
            # model_kick.save(os.path.join(save_dir, f'model_kick_episode_{i_episode}.h5')) 
            rewards=[0]
            values=[0]
            break

    rewards=[0]
    values=[0]
    
env.close()
pygame.quit()

1/1 [==============================] - 0s 74ms/step - loss: 19.5511 - softmax_loss: 0.2000 - value_layer_loss: 19.3511
[[ 17.761433  18.230886 239.21991  169.88594 ]]
1/1 [==============================] - 0s 74ms/step - loss: 2442.5862 - softmax_loss: 0.2219 - value_layer_loss: 2442.3643
[[ 14.973888  17.772873 187.17488  165.85973 ]]
1/1 [==============================] - 0s 73ms/step - loss: 6954.6919 - softmax_loss: 0.2205 - value_layer_loss: 6954.4712
[[ 15.644356  19.741581 197.21606  158.88261 ]]
1/1 [==============================] - 0s 471ms/step - loss: 7007.0864 - dense_2_loss: 0.0000e+00 - value_layer_loss: 7007.0864
[[ 19.630825  19.996641 307.5508   179.54059 ]]
1/1 [==============================] - 0s 73ms/step - loss: 7061.8364 - dense_2_loss: 0.0000e+00 - value_layer_loss: 7061.8364
[[ 19.938763  19.999996 334.12927  136.74594 ]]
1/1 [==============================] - 0s 74ms/step - loss: 6857.5044 - dense_2_loss: 0.0000e+00 - value_layer_loss: 6857.5044
[[ 19.988798 

KeyboardInterrupt: 

In [ ]:
rewards = [0]
values = [0]
episodes = 20

# Shared arrays for tensorboard
episode_rewards        = []
episode_lengths        = []
episode_mean_values    = []
episode_invalid_ops    = []
for i_episode in range(episodes):
    valid_actions          = [[0,1,2],[0,1,2]]
    episode_reward = episode_step_count = episode_inv_count = 0
    # Reset the state
    state = env._set_world()
    actions = []
    for i in range(100):              
        move, value = model_move.predict([state[0],state[1]])
        move = move.reshape((2,3))
        move = np.argmax(move, axis = 1)                   
        # Execute action and observe
        next_state, reward, done, valid_actions, info = env.step(move,render=True)
        # next_state, reward, done, valid_actions, info = env.step(actions,render=False)   
        rewards.append(rewards[-1] + reward)  
        values.append(values[-1] + value[0,0])          
        # Here we take the rewards and values from the rollout, and use them to 
        # generate the advantage and discounted returns. (With bootstrapping)
        # The advantage function uses "Generalized Advantage Estimation"
        rewards_plus = np.asarray(rewards + [bootstrap_value])
        discounted_rewards = discount(rewards_plus,gamma)[:-1]        
        value_plus = np.asarray(values + [bootstrap_value])
        advantages = rewards + gamma * value_plus[1:] - value_plus[:-1]
        advantages = discount(advantages,gamma)
        # value_plus = np.expand_dims(value_plus, axis=-1)
        # value_plus = np.expand_dims(value_plus, axis=0)
        # Debugging information to check shapes
        advantages = np.expand_dims(advantages, axis = 0)        
        move = tf.one_hot(actions[:2],3)
        move = np.expand_dims(move, axis=0)
        # print(state[0].shape)
        # Train the model with the updated Q-value
        model_move.fit([state[0],state[1]], [move,advantages]) 
        # Update state
        state = next_state

        if done:            
            rewards=[0]
            values=[0]
            break

    rewards=[0]
    values=[0]

In [ ]:
# Create subplots
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)

# Plot rewards in the first subplot
ax1.plot(rewards)
ax1.set_title('Rewards')

# Plot values in the second subplot
ax2.plot(values)
ax2.set_title('Values')

# Add labels, legends, or other customization as needed

# Show the plot
plt.show()

In [ ]:
obs = env._observe(0)
plt.imshow(obs[0])
print(obs[2])